In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word_to_ix = {"salam": 0, "necəsən": 1}
embeds = nn.Embedding(2, 5)  
lookup_tensor = torch.tensor([word_to_ix["necəsən"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.2673, -0.4212, -0.5107, -1.5727, -0.1232]],
       grad_fn=<EmbeddingBackward0>)


In [7]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [6]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [8]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [9]:
for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

      
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

     
        model.zero_grad()

      
        log_probs = model(context_idxs)

       
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

      
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  


print(model.embeddings.weight[word_to_ix["beauty"]])

[516.8106060028076, 514.3368418216705, 511.87966799736023, 509.4371349811554, 507.0097324848175, 504.5950999259949, 502.19280886650085, 499.8031039237976, 497.42474818229675, 495.05787801742554]
tensor([-0.2692,  1.3117, -0.8906, -0.7521, -0.0578,  2.2006, -0.7550, -0.4346,
         0.6704,  0.6505], grad_fn=<SelectBackward0>)


In [10]:
CONTEXT_SIZE = 2  
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()


vocab = set(raw_text)
vocab_size = len(vocab)


In [11]:
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [12]:
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['are', 'We', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]


-------------------------------------

In [70]:
import unicodedata
import string

In [71]:
all_letters = string.ascii_letters + " .,;'"

In [78]:
def unicodeToAscii(s):
    s =s.lower().replace('ə','e')
    s =s.replace('ı','i')
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [79]:
print(unicodeToAscii('günöğəəğqeş'))

gunogeegqes


In [80]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

CONTEXT_SIZE = 2 
EMDEDDING_DIM = 100

raw_text = """Mənim halım dəyişdi, ayaqlarım taqətdən düşdü. 
Parçalanmış paroxodda bu quldur dəstəsilə qalmaq dəhşət idi. 
Ancaq durub gözləmək, fikirləşmək vaxtı deyildi. İndi bu qayıq 
bizim özümüzə lazım idi, odur ki, biz hər nə cür olursa olsun 
həmin qayığı tapmalı idik. Biz paroxodun sağ tərəfi ilə getməyə 
başladıq, titrəyib əsə-əsə, güc-bəla ilə dal tərəfə gəlib çatdıq; 
mənə elə gəlirdi ki, buraya çatıncayadək azı bir həftə keçdi. 
Qayıqdan heç bir nişanə yox idi. Cim dedi ki, o deyəsən daha gedə 
bilməyəcək, qorxudan lap taqətini itirib, tamam gücdən düşüb. 
Mən ona bildirdim ki, necə olursa olsun getmək lazımdır, əgər 
burada qalsaq işimiz pis olacaq, bu lap yəqin m




Biz yollandıq. Kapitan kayutunun dal tərəfini axtarmağa başladıq, 
tapdıq, sonra gücümüzü toplayıb qaranlıqda, gözəyarı paroxodun 
kənarlarındakı çıxıntılardan tuta-tuta irəlilədik, çünki onun bir 
tərəfı artıq suyun içində idi, beləliklə, bil losman budkasının 
şüşəbəndinə çatdıq. Qapıya çatan kimi gördük ki, qayıq buradadır. 
Mən onu qaranlıqda güclə görmüşdüm. Sevindiyimdən bilmirdim nə edim! 
Bir saniyə də keçsəydi mən onun içində olacaqdım, ancaq bu vaxt qapı açıldı.""".split()

for ix,word in enumerate(raw_text):
    raw_text[ix] = unicodeToAscii(word)
    
vocab = set(raw_text)

vocab_size = len(vocab)

word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

In [83]:
data[1]

(['halim', 'deyisdi,', 'taqetden', 'dusdu.'], 'ayaqlarim')

In [84]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

      
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
       
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

In [85]:
model = CBOW(vocab_size, EMDEDDING_DIM)

In [86]:
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [87]:
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

   
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()


In [98]:
context = ['men','ona','ki,', 'nece']
for ix,word in enumerate(context):
    context[ix] = unicodeToAscii(word)
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: olursa hemin tapmali idik. taqetden dusdu. parcalanmis paroxodda bu quldur destesile qalmaq dehset idi. ancaq durub gozlemek, fikirlesmek vaxti deyildi. indi bu qayiq bizim ozumuze lazim idi, odur ki, biz her ne cur olursa olsun hemin qayigi tapmali idik. biz paroxodun sag terefi ile getmeye basladiq, titreyib eseese, gucbela ile dal terefe gelib catdiq; mene ele gelirdi ki, buraya catincayadek azi bir hefte kecdi. qayiqdan hec bir nisane yox idi. cim dedi ki, o deyesen daha gede bilmeyecek, qorxudan lap taqetini itirib, tamam gucden dusub. men ona bildirdim ki, nece olursa olsun getmek lazimdir, eger burada qalsaq isimiz pis olacaq, bu lap yeqin meseledir. biz yollandiq. kapitan kayutunun dal terefini axtarmaga basladiq, tapdiq, sonra gucumuzu toplayib qaranliqda, gozeyari paroxodun kenarlarindaki cixintilardan tutatuta ireliledik, cunki onun bir terefi artiq suyun icinde idi, belelikle, bil losman budkasinin susebendine catdiq. qapiya catan kimi gorduk ki, qayiq buradadir. 

In [102]:
context = ['olursa','həmin','tapmali', 'idik.']
for ix,word in enumerate(context):
    context[ix] = unicodeToAscii(word)
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: olursa hemin tapmali idik. taqetden dusdu. parcalanmis paroxodda bu quldur destesile qalmaq dehset idi. ancaq durub gozlemek, fikirlesmek vaxti deyildi. indi bu qayiq bizim ozumuze lazim idi, odur ki, biz her ne cur olursa olsun hemin qayigi tapmali idik. biz paroxodun sag terefi ile getmeye basladiq, titreyib eseese, gucbela ile dal terefe gelib catdiq; mene ele gelirdi ki, buraya catincayadek azi bir hefte kecdi. qayiqdan hec bir nisane yox idi. cim dedi ki, o deyesen daha gede bilmeyecek, qorxudan lap taqetini itirib, tamam gucden dusub. men ona bildirdim ki, nece olursa olsun getmek lazimdir, eger burada qalsaq isimiz pis olacaq, bu lap yeqin meseledir. biz yollandiq. kapitan kayutunun dal terefini axtarmaga basladiq, tapdiq, sonra gucumuzu toplayib qaranliqda, gozeyari paroxodun kenarlarindaki cixintilardan tutatuta ireliledik, cunki onun bir terefi artiq suyun icinde idi, belelikle, bil losman budkasinin susebendine catdiq. qapiya catan kimi gorduk ki, qayiq buradadir. 

In [106]:
context = ['bu','quldur','tapmali', 'idik.']
for ix,word in enumerate(context):
    context[ix] = unicodeToAscii(word)
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: olursa hemin tapmali idik. taqetden dusdu. parcalanmis paroxodda bu quldur destesile qalmaq dehset idi. ancaq durub gozlemek, fikirlesmek vaxti deyildi. indi bu qayiq bizim ozumuze lazim idi, odur ki, biz her ne cur olursa olsun hemin qayigi tapmali idik. biz paroxodun sag terefi ile getmeye basladiq, titreyib eseese, gucbela ile dal terefe gelib catdiq; mene ele gelirdi ki, buraya catincayadek azi bir hefte kecdi. qayiqdan hec bir nisane yox idi. cim dedi ki, o deyesen daha gede bilmeyecek, qorxudan lap taqetini itirib, tamam gucden dusub. men ona bildirdim ki, nece olursa olsun getmek lazimdir, eger burada qalsaq isimiz pis olacaq, bu lap yeqin meseledir. biz yollandiq. kapitan kayutunun dal terefini axtarmaga basladiq, tapdiq, sonra gucumuzu toplayib qaranliqda, gozeyari paroxodun kenarlarindaki cixintilardan tutatuta ireliledik, cunki onun bir terefi artiq suyun icinde idi, belelikle, bil losman budkasinin susebendine catdiq. qapiya catan kimi gorduk ki, qayiq buradadir. 